In [1]:
using DataFrames, CSV
using JuMP, GLPKMathProgInterface

In [2]:
function str2vec(str)
    S = split(str, " ");
    V = [parse(Int, s) for s in S];
end;

function vec2str(vec)
    str = join(vec, " ");
end;

In [39]:
# Open schedule info
df_tensors = DataFrame(CSV.File("segmentation/tensor_details.csv"));
df_schedule = DataFrame(CSV.File("segmentation/execution_schedule_info.csv"));

fixed_tensors = [183, 179]

no_tensors = size(df_tensors, 1);
no_operators = size(df_schedule, 1);

df_tensors.Idx = 1:no_tensors;

MAX_MEM = 7 * 1024 * 1024;
MIN_CH = 8;
MAX_CH = 64 * 32;

In [40]:
names(df_tensors)

6-element Array{String,1}:
 "Id"
 "Name"
 "Shape"
 "Size"
 "FixedTo"
 "Idx"

In [41]:
opt_model = Model(solver=GLPKSolverMIP());

In [42]:
@variable(opt_model, x[1:no_tensors], Int);

opt_model

Feasibility problem with:
 * 0 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [43]:
# Set max channels for other tensors
nfixed_df = df_tensors[.!in.(df_tensors.Id, Ref(fixed_tensors)), :];
for r in eachrow(nfixed_df)
    if r.FixedTo !== missing
        fixedto = str2vec(r.FixedTo);
        fixedto_df = df_tensors[in.(df_tensors.Id, Ref(fixedto)), :];
        fcon = 0;
        for rf in eachrow(fixedto_df)
            fcon = fcon + x[rf.Idx];
        end
        if fcon != 0
            @constraint(opt_model, x[r.Idx] == fcon);
        end
    else
        @constraint(opt_model, x[r.Idx] <= MAX_CH);
        # @constraint(opt_model, x[r.Idx] >= MIN_CH);
        
        shape = str2vec(r.Shape);
        @constraint(opt_model, x[r.Idx] >= shape[4]);
    end
end

opt_model

Feasibility problem with:
 * 118 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [44]:
# Set fixed tensor constraints
fixed_df = df_tensors[in.(df_tensors.Id, Ref(fixed_tensors)), :]
for i in 1:length(fixed_tensors)
    idx = fixed_df[i, :].Idx;
    size = fixed_df[i, :].Size;
    shape = str2vec(fixed_df[i, :].Shape);
    feat = shape[1] * shape[2] * shape[3];
    
    @constraint(opt_model, x[idx] == shape[4]);
    @constraint(opt_model, feat * x[idx] <= MAX_MEM);
end
opt_model

Feasibility problem with:
 * 122 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [45]:
# Set constraints
for s in 1:no_operators
    tensors = str2vec(df_schedule[s, 2])
    sel = df_tensors[in.(df_tensors.Id, Ref(tensors)), :];
    c = 0;
    for r in eachrow(sel)
        if r.Id in fixed_tensors
            c = c + x[r.Idx];
        else
            shape = str2vec(r.Shape);
            feat = shape[1] * shape[2] * shape[3];
            c = c + feat * x[r.Idx];
        end
        
        if c != 0
            @constraint(opt_model, c <= MAX_MEM);
        end
    end
end

opt_model

Feasibility problem with:
 * 296 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [46]:
# Set cost function
@objective(opt_model, Max, sum(x));

opt_model

Maximization problem with:
 * 296 linear constraints
 * 71 variables: 71 integer
Solver is GLPKMathProgInterface.GLPKInterfaceMIP.GLPK

In [47]:
status = solve(opt_model)

:Optimal

In [48]:
print(getvalue(x))

[2048.0, 425.0, 16.0, 393.0, 1158.0, 48.0, 579.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 579.0, 579.0, 579.0, 579.0, 2048.0, 1158.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 2048.0, 21.0, 21.0, 1322.0, 3370.0, 2048.0, 2048.0, 21.0, 3.0]

In [49]:
x_ch = getvalue(x);
df_tensors_out = copy(df_tensors);
df_tensors_out[!, "NewShape"] .= "1";
for r in eachrow(df_tensors_out)
    shape = str2vec(r.Shape);
    shape[4] = x_ch[r.Idx];
    shape_str = vec2str(shape);
    r.NewShape = shape_str;
end

CSV.write("segmentation/tensor_details_out.csv", df_tensors_out);

df_tensors_out

,Id,Name,Shape
,Int64,String,String
1,0,AvgPool2D/AvgPool,1 1 1 160
2,2,MobilenetV2/Conv/Relu6,1 129 129 16
3,4,MobilenetV2/expanded_conv/depthwise/Relu6,1 129 129 16
4,7,MobilenetV2/expanded_conv/project/BatchNorm/FusedBatchNorm,1 129 129 8
5,10,MobilenetV2/expanded_conv_1/depthwise/Relu6,1 65 65 48
6,14,MobilenetV2/expanded_conv_1/expand/Relu6,1 129 129 48
7,16,MobilenetV2/expanded_conv_1/project/BatchNorm/FusedBatchNorm,1 65 65 12
8,19,MobilenetV2/expanded_conv_10/depthwise/Relu6,1 33 33 192
9,23,MobilenetV2/expanded_conv_10/expand/Relu6,1 33 33 192
